In [2]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import Ridge

# Dataset concaténés de chaque WF

X_train_tot = pd.read_csv('X_train_v2.csv', sep=',', index_col='ID', header=0)
Y_train_tot = pd.read_csv('Y_train_sl9m6Jh.csv', sep=',', index_col='ID', header=0)

for i in range(6):
    X_train_tot[X_train_tot['WF']==('WF'+str(i))]=X_train_tot.loc[X_train_tot['WF']==('WF'+str(i))].interpolate()

X_test_tot = pd.read_csv('X_test_v2.csv', sep=',', index_col='ID', header=0)
Y_test_tot = pd.read_csv('Y_test_random.csv', sep=',', index_col='ID', header=0)


# On récupère les data spécifiques à chaque WF
# WF_X_train[i] contient alors le DataFrame de l'éolienne i

WF_X_train = [None]
WF_Y_train = [None]

WF_X_test = [None]
WF_Y_test = [None]

WF_X_train.append(X_train_tot.loc[X_train_tot['WF']==('WF1')])
WF_Y_train.append(Y_train_tot.iloc[:WF_X_train[1].shape[0]])

WF_X_test.append(X_test_tot.loc[X_test_tot['WF']==('WF1')])
WF_Y_test.append(Y_test_tot.iloc[:WF_X_test[1].shape[0]])

k = 0

for i in range(1,6):
    WF_X_train.append(X_train_tot.loc[X_train_tot['WF']==('WF'+str(i+1))])
    k += WF_X_train[i].shape[0]
    WF_Y_train.append(Y_train_tot.iloc[k:k+WF_X_train[i+1].shape[0]])

k = 0

for i in range(1,6):
    WF_X_test.append(X_test_tot.loc[X_test_tot['WF']==('WF'+str(i+1))])
    k += WF_X_test[i].shape[0]
    WF_Y_test.append(Y_test_tot.iloc[k:k+WF_X_test[i+1].shape[0]])
    

# Maintenant il nous faut récupérer les données des différentes stations, pour cela on procède de la même manière
# On doit identifier le numéro dans le nom de la colonne NWPi..

# On parcourt les différentes éoliennes

for i in range(1,len(WF_X_train)) :
    extraction_stations = [None]
    WFi = WF_X_train[i]
    # On traite chaque numéro de station
    k=1
    # On parcourt les données de gauche à droite en extrayant les données relatives à la station k entre n1 et n2
    n1 = 2 # On ne prend pas les colonnes WF et Time, on les rajoutera après
    n2 = 3 
    while k < 5 :
        # Si k vaut 4, alors on prend tous les derniers éléments
        if k == 4 :
            n2 = WFi.shape[1]-1
            donnees_station = WFi.iloc[:,n1:n2+1]
            
        # Si k ne vaut pas 4 alors on incrémente n2 librement jusqu'à passer à une nouvelle station
        else :
            nom_col = WFi.columns[n2]
            while int(nom_col.split('NWP')[1].split('_')[0]) == k :
                n2 +=1
                nom_col = WFi.columns[n2]
            
            #On extrait les données
            donnees_station = WFi.iloc[:,n1:n2]

        #On insert le temps
        donnees_station.insert(0, 'Time', WFi.iloc[:,1])
        extraction_stations.append(donnees_station)
        
        n1 = n2
        n2 += 1
        k += 1
        
    WF_X_train[i] = extraction_stations
            

for i in range(1,len(WF_X_test)) :
    extraction_stations = [None]
    WFi = WF_X_test[i]
    # On traite chaque numéro de station
    k=1
    # On parcourt les données de gauche à droite en extrayant les données relatives à la station k entre n1 et n2
    n1 = 2 # On ne prend pas les colonnes WF et Time, on les rajoutera après
    n2 = 3 
    while k < 5 :
        # Si k vaut 4, alors on prend tous les derniers éléments
        if k == 4 :
            n2 = WFi.shape[1]-1
            donnees_station = WFi.iloc[:,n1:n2+1]
            
        # Si k ne vaut pas 4 alors on incrémente n2 librement jusqu'à passer à une nouvelle station
        else :
            nom_col = WFi.columns[n2]
            while int(nom_col.split('NWP')[1].split('_')[0]) == k :
                n2 +=1
                nom_col = WFi.columns[n2]
            
            #On extrait les données
            donnees_station = WFi.iloc[:,n1:n2]

        #On insert le temps
        donnees_station.insert(0, 'Time', WFi.iloc[:,1])
        extraction_stations.append(donnees_station)
        
        n1 = n2
        n2 += 1
        k += 1
        
    WF_X_test[i] = extraction_stations
            


# WF_X_train[i][j] contient les données de l'éolienne i pour la station j
# WF_Y_train[i] contient les données resultats correspondants à l'éolienne i

# Idem pour les test

In [3]:
def separation(df):
    #df_U=pd.DataFrame(columns=[])
    #Création de 4 dataframe vierge qui prennent le premier argument de Df qui mets la 1 ere colonne de df
    df_U=df.iloc[:,[0]]
    df_V=df.iloc[:,[0]]
    df_T=df.iloc[:,[0]]
    df_CLCT=df.iloc[:,[0]]
    #nom colonne=liste de toute les collones
    Nom_Colonne=list(df)
    for Colonne in Nom_Colonne:
        if '_U'in Colonne:
            df_U = pd.merge(df_U, df[[Colonne]], on='ID')  
        if '_V' in Colonne:
            df_V = pd.merge(df_V, df[[Colonne]], on='ID')
        if '_T' in Colonne:
            df_T = pd.merge(df_T, df[[Colonne]], on='ID')
        if '_CLCT' in Colonne:
            df_CLCT = pd.merge(df_CLCT, df[[Colonne]], on='ID')
    return(df_U,df_V,df_T,df_CLCT)

def valeurs_finales(df):
    
    [df_U,df_V,df_T,df_CLCT]=separation(df)
    
    df_U=df_U.fillna(method='ffill',axis=1)
    df_V=df_V.fillna(method='ffill',axis=1)
    df_T=df_T.fillna(method='ffill',axis=1)
    df_CLCT=df_CLCT.fillna(method='ffill',axis=1)
    df_final=df.iloc[:,[0]]
    df_final = pd.merge(df_final, df_U.iloc[:,[-1]], on='ID') 
    df_final = pd.merge(df_final, df_V.iloc[:,[-1]], on='ID')

    if len(list(df_T))>1:
        df_final = pd.merge(df_final, df_T.iloc[:,[-1]], on='ID')
    if len(list(df_CLCT))>1:
        df_final = pd.merge(df_final, df_CLCT.iloc[:,[-1]], on='ID')
    df_final=df_final.dropna()
    Liste_compteur=[]
    for x in df_final.itertuples():
        compteur=None
        for k in range (len(list(df_final))):
            if k>0 and x[k]!=x[1]:
                compteur=1
        Liste_compteur.append(compteur)
    df_final['Compteur']=Liste_compteur

    df_final=df_final.dropna()
    del df_final['Compteur']
    return(df_final)


#separation(df) sert à séparer un dataframe en 2,3 ou 4 dataframe avec les données de U, V, T et CLCT quand elles y sont
#dernieres_valeurs(df) donne les dernieres valeurs mesurées de u,v,t,et clct quand elles existent du dataframe d'entrée, fonctionne qq soit WF ou NWP

In [4]:
# On fait une concaténation de la WF[i]

def concatene(i,df):
    df_1=valeurs_finales(df[i][1])
    df_2=valeurs_finales(df[i][2])
    df_3=valeurs_finales(df[i][3])
    df_4=valeurs_finales(df[i][4])
    df_final = df[i][1].iloc[:,[0]]
    df_final = pd.merge(df_final, df_1.iloc[:,1:], on='ID', how='outer')
    df_final = pd.merge(df_final, df_2.iloc[:,1:], on='ID', how='outer')
    df_final = pd.merge(df_final, df_3.iloc[:,1:], on='ID', how='outer')
    df_final = pd.merge(df_final, df_4.iloc[:,1:], on='ID', how='outer')
    df_final = df_final.fillna(method="bfill")
    df_final = df_final.fillna(method="ffill")
    del df_final['Time']
    df_final.columns=['NWP1_U','NWP1_V','NWP1_T','NWP2_U','NWP2_V','NWP3_U','NWP3_V','NWP3_T','NWP4_U','NWP4_V','NWP4_CLCT']
    return(df_final)
    
#concatene(i) prend en entrée le numéro de la WF et sors un dataframe avec les principales valeurs pour chaque station météo. 
#les données manquantes sont complété par les valeurs précédentes, ca peut etre améliorer (en faisant qqc de linéaire)

In [5]:
# fonction de normalisation des données : prend un dataframe et le normalise par rapport à chacune de ses colonnes
#attention: il ne doit pas y avoir de blancs

def normalisation(df_init):
    df=df_init.copy()
    if 'Time' in list(df):
        del(df['Time'])
    if 'WF' in list(df):
        del(df['WF'])
    X_scaled = pd.DataFrame(preprocessing.scale(df))
    return(X_scaled)
    

#pour linstant n'arrive pas à merge car normalise aussi ID
#pas besoin, ca le fait dans les arguments de ridge

In [6]:
#faire une régression ridge
def regression_ridge(X,y,alpha):
    clf = Ridge(alpha=1.0, normalize=True)
    clf.fit(X, y)

#fonctionne mais sert à rien: lappliquer directement

In [7]:
# Retourne la norme et la direction du vent pour une valeur de u et une valeur de v en entrée
def calcule_force_direction(u,v):
    
    force = np.sqrt(u**2 + v**2)
    
    if u > 0 :
        if v < 0 :
            direction = np.arctan(v/u) + 2*np.pi
            
        elif v>=0 :
            direction = np.arctan(v/u)
    
    elif u < 0 :
        if v < 0 :
            direction = np.arctan(v/u) + np.pi
            
        elif v>=0 :
            direction = np.arctan(v/u) + np.pi
    
    elif u == 0 :
        if v > 0 :
            direction = np.pi/2
        
        if v< 0 :
            direction = 3/2 * np.pi/2
        
        if v == 0 :
            direction = 0
    
    return(force, direction)
    

# Remplace les valeurs de U,V du dataframe WF_X_train ou WF_X_test en argument par la norme du vent et sa direction
def force_direction(WF_X):
    # Pour chaque éolienne on parcourt les données, on crée deux array direction et force, on parcourt chaque ligne
    # on ajoute la bonne valeur dans l'array. Une fois ceci fait, on supprime les colonnes U et V, on ajoute les colonnes 
    # direction et force.
    
    WF = [None]
    
    for i in range(1,7):
        
        # Séparation des différentes données, U,V et T, et calcul des moyennes sur une unique colonne
        df = concatene(i, WF_X)
        U1 = df.loc[:,['NWP1_U','NWP2_U','NWP3_U']].mean(axis = 1)
        V1 = df.loc[:,['NWP1_V','NWP2_V','NWP3_V']].mean(axis = 1)
        T = df.loc[:,['NWP1_T','NWP3_T']].mean(axis = 1)
            
        U2 = df.loc[:,'NWP4_U']
        V2 = df.loc[:,'NWP4_V']
        CLCT = df.loc[:,'NWP4_CLCT']
        

        # Calcul du sens et de la direction du vent
        force1 = []
        direction1 = []
        
        force2=[]
        direction2=[]
        
        for j in range(df.shape[0]):
            u1 = U1.iloc[j]
            v1 = V1.iloc[j]
            u2 = U2.iloc[j]
            v2 = V2.iloc[j]
            
            [f1,d1] = calcule_force_direction(u1,v1)
            [f2,d2] = calcule_force_direction(u2,v2)
            
            force1.append(f1)
            direction1.append(d1)
            force2.append(f2)
            direction2.append(d2)
        
        force1 = np.array(force1)
        force2 = np.array(force2)
        direction1 = np.array(direction1)
        direction2 = np.array(direction2)
        
        new_dataframe = pd.DataFrame({'NWP123_T':T, 'NWP123_force':force1, 'NWP123_dir':direction1, 'NWP_4_force':force2, 'NWP4_dir':direction2, 'NWP4_CLCT':CLCT }) 
        
        
        
        WF.append(new_dataframe)
    
    return(WF)

def coordonnees_norme(WF_X):
    # Pour chaque éolienne on parcourt les données, on crée deux array direction et force, on parcourt chaque ligne
    # on ajoute la bonne valeur dans l'array. Une fois ceci fait, on supprime les colonnes U et V, on ajoute les colonnes 
    # direction et force.
    
    WF = [None]
    
    for i in range(1,7):
        
        # Séparation des différentes données, U,V et T, et calcul des moyennes sur une unique colonne
        df = concatene(i, WF_X)
        U1 = df.loc[:,['NWP1_U','NWP2_U','NWP3_U']].mean(axis = 1)
        V1 = df.loc[:,['NWP1_V','NWP2_V','NWP3_V']].mean(axis = 1)
        T = df.loc[:,['NWP1_T','NWP3_T']].mean(axis = 1)
            
        U2 = df.loc[:,'NWP4_U']
        V2 = df.loc[:,'NWP4_V']
        CLCT = df.loc[:,'NWP4_CLCT']
        

        # Calcul du sens et de la direction du vent
        norme1 = []
        
        norme2 =[]
        
        for j in range(df.shape[0]):
            u1 = U1.iloc[j]
            v1 = V1.iloc[j]
            u2 = U2.iloc[j]
            v2 = V2.iloc[j]
            
            n1 = np.sqrt(u1**2 + v1**2)
            n2 = np.sqrt(u2**2 + v2**2)
            
            norme1.append(n1)
            norme2.append(n2)

        
        norme1 = np.array(norme1)
        norme2 = np.array(norme2)
        
        new_dataframe = pd.DataFrame({'NWP123_T':T, 'NWP123_U':U1 , 'NWP123_V':V1, 'NWP123_norme':norme1, 'NWP4_U':U2, 'NWP4_V':V2, 'NWP4_norme':norme2, 'NWP4_CLCT':CLCT }) 
        
        WF.append(new_dataframe)
    
    return(WF)


def sans_T(df):
    for x in df.columns:
        if '_T' in x :
            df = df.drop([x], axis=1)
    return(df)

In [8]:
# Entrainement de l'algorithme, et prédiction des valeurs de y_test, puis enregistrement du fichier pour submition
from sklearn.ensemble import RandomForestRegressor


WF_y_pred = []
for i in range(1,7):
    
    # Récupération des données d'entrainement
    X_train = concatene(i,WF_X_train)
    y_train = np.ravel(WF_Y_train[i])
    
    # Définition de l'algorithme
    regressor = RandomForestRegressor(n_estimators=1000)
    
    # Entrainement
    regressor.fit(X_train, y_train)
    
    # Préparation des données de test et prédiction
    X_test = concatene(i,WF_X_test)
    y_pred = regressor.predict(X_test)
    
    WF_y_pred.append(y_pred)

    
# Formation d'un unique vecteur y_pred
y_pred = np.array([])
for i in range(6):
    y_pred = np.hstack((y_pred,WF_y_pred[i]))

# Mise en forme du fichier
# Crée un dataframe vide, créer une colonne prenant les valeur de y_pred, réindex selon les valeurs attendues pour submit

df = pd.DataFrame()
df['Production'] = y_pred
index_start = 37376
df = df.rename(index= lambda s: s + index_start)

# Enregistre le dataframe dans un csv coma-separated du nom de y_pred.csv dans le repertoire de travail.
df.to_csv('y_pred.csv')

In [9]:
# Entrainement de l'algorithme, puis prédiction des valeurs de y_train, et calcul de l'erreur (pour test)

from sklearn.ensemble import RandomForestRegressor

WF_y_pred = []

# Toujours plus de noms de variables
WF_X_train_tot = force_direction(WF_X_train)

for i in range(1,7):
    y_train = np.ravel(WF_Y_train[i])
    
    X_train = WF_X_train_tot[i]
    
    regressor = RandomForestRegressor(n_estimators=1000, random_state=0, oob_score=True)
    
    regressor.fit(X_train, y_train)
    
    y_pred = regressor.predict(X_train)
    
    WF_y_pred.append(y_pred)

    
# Formation d'un unique vecteur y_pred
y_pred = np.array([])

for i in range(6):
    
    y_pred = np.hstack((y_pred,WF_y_pred[i]))

    
# Mise en forme de y_train et calcul de l'erreur de prédiction sur le set de test.

y_train = np.ravel(Y_train_tot)

from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_train, y_pred)

0.1754528449530183